<a href="https://colab.research.google.com/github/ishancoderr/Neural-networks/blob/main/Boltzman_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q torch==1.0.0 torchvision


     |████████████████████████████████| 591.8MB 33kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.0.0 which is incompatible.


In [4]:
import torch
print(torch.__version__)

1.0.0


In [5]:
!pip install torch

In [6]:
import numpy as np
import pandas as pd


In [7]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
